In [1]:
# first neural network with keras tutorial
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow as tf

import numpy as np
import pandas as pd
import os
import PIL
import PIL.Image
import tensorflow as tf

#from sklearn.model_selection import train_test_split

In [2]:
train_ds = keras.utils.image_dataset_from_directory(
    'data/',
    validation_split=0.2,
    subset="training",
    image_size=(28, 28),
    seed=132,
    batch_size=32
)

Found 10000 files belonging to 10 classes.
Using 8000 files for training.


In [3]:
val_ds = keras.utils.image_dataset_from_directory(
    'data/',
    validation_split=0.2,
    subset="validation",
    image_size=(28, 28),
    seed=132,
    batch_size=32
)

Found 10000 files belonging to 10 classes.
Using 2000 files for validation.


In [4]:
# checking class names

print(train_ds.class_names)

['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']


In [5]:
for image_batch, labels_batch in train_ds:
  print(image_batch.shape)
  print(labels_batch.shape)
  break

(32, 28, 28, 3)
(32,)


In [6]:
# standardize layer

normalization_layer = tf.keras.layers.Rescaling(1./255)

In [7]:
# configure for performance

AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [8]:
model = tf.keras.Sequential([
  tf.keras.layers.Rescaling(1./255),

  tf.keras.layers.Conv2D(32, 3, activation='relu'),
  tf.keras.layers.MaxPooling2D(),
  
  tf.keras.layers.Conv2D(32, 3, activation='relu'),
  tf.keras.layers.MaxPooling2D(),

  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(40, activation='relu'),
  tf.keras.layers.Dense(40, activation='relu'),
  tf.keras.layers.Dense(10, activation='softmax')
])

In [9]:
model.compile (
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

In [11]:
model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=50
)

Epoch 1/50
250/250 [==============================] - 1s 4ms/step - loss: 2.3026 - accuracy: 0.1921 - val_loss: 2.3026 - val_accuracy: 0.1735
Epoch 2/50
250/250 [==============================] - 1s 4ms/step - loss: 2.3026 - accuracy: 0.1921 - val_loss: 2.3026 - val_accuracy: 0.1735
Epoch 3/50
250/250 [==============================] - 1s 4ms/step - loss: 2.3026 - accuracy: 0.1921 - val_loss: 2.3026 - val_accuracy: 0.1735
Epoch 4/50
250/250 [==============================] - 1s 4ms/step - loss: 2.3026 - accuracy: 0.1921 - val_loss: 2.3026 - val_accuracy: 0.1735
Epoch 5/50
250/250 [==============================] - 1s 4ms/step - loss: 2.3026 - accuracy: 0.1921 - val_loss: 2.3026 - val_accuracy: 0.1735
Epoch 6/50
250/250 [==============================] - 1s 4ms/step - loss: 2.3026 - accuracy: 0.1921 - val_loss: 2.3026 - val_accuracy: 0.1735
Epoch 7/50
250/250 [==============================] - 1s 4ms/step - loss: 2.3026 - accuracy: 0.1921 - val_loss: 2.3026 - val_accuracy: 0.1735
Epoch 

KeyboardInterrupt: 